In [1]:
import sys
sys.path.append("../script/")

import Functions
from Engine import Engine
from GTGP import GTGP


from sklearn.metrics import classification_report,accuracy_score,confusion_matrix,f1_score
from sklearn.model_selection import train_test_split
import numpy as np
import importlib
from time import time
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import numpy as np
import pandas as pd
from time import time

In [2]:
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.utils import shuffle
def plot_roc_curve(true_y, y_prob):
    """
    plots the roc curve based of the probabilities
    """
    
    fpr, tpr, thresholds = roc_curve(true_y, y_prob)
    plt.plot(fpr, tpr)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    print(roc_auc_score(true_y,prob[:,1]))

# data

In [4]:
df = pd.read_csv("../data/confidence.tsv",delimiter='\t')
X = df.iloc[:,:-1].to_numpy().astype("float")
y = df.iloc[:,-1].to_numpy().astype("int")

seeds = [10086, 200,500,30506,30405,30420,10056,7059,40965,5398,869543,83491,823190,
         48392,2810,48392,3498210,483902,859032,12890,538920,86954,54309,6504,9840,
         219805,548,2981,432890,5438908,219094,5843902,60854,979,12890,2108,4093]
train_size = 0.7
dataset = 'confidence'

In [9]:
def to_one_hot(y,yt):
    le = LabelEncoder()
    ohe = OneHotEncoder()
    y_train = le.fit_transform(y)
    y_one_hot = ohe.fit_transform(y_train.reshape(-1,1))
    
    y_test = le.transform(yt)
    yt_one_hot = ohe.transform(y_test.reshape(-1,1))
    
    return y_train,y_test,y_one_hot,yt_one_hot

# model

In [28]:
def fit_trees():
    learning_rate=0.1
    max_depth=5
    bins=8
    lam=100

    gtgp = GTGP(learning_rate=learning_rate,max_depth=max_depth,bins=bins,lam=lam)

    total_size=10
    elite_size = 10
    epoch= 100
    gp_epoch= 3
    verbose = 1
    tolerance=0.5

    gtgp.fit(X_train,y_train,total_size=total_size,elite_size = elite_size,epoch=epoch,gp_epoch=gp_epoch,tolerance=tolerance,verbose=verbose)

    retrain_epoch=20
    alpha=0
    beta=1
    gammer=0

    verbose=1
    gtgp.lam = 10
    gtgp.retrain_estimators(X_test,y_test,retrain_epoch=retrain_epoch,alpha=alpha,beta=beta,gammer=gammer,verbose=verbose)

    return gtgp

for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
    gtgp = fit_trees()

    print(classification_report(y_train,np.argmax(gtgp.train_p,axis=1),digits=5))
    print(classification_report(y_test,np.argmax(gtgp.test_p,axis=1),digits=5))

    num_trees,depth,num_nodes = gtgp.print_model()

    train_acc = accuracy_score(y_train,np.argmax(gtgp.train_p,axis=1))
    test_acc = accuracy_score(y_test,np.argmax(gtgp.test_p,axis=1))
    train_f1 = f1_score(y_train,np.argmax(gtgp.train_p,axis=1),average='macro')
    test_f1 = f1_score(y_test,np.argmax(gtgp.test_p,axis=1),average='macro')


    with open('./benchmark/'+dataset+'.csv','a') as f:
        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

5 19 1
6 18 2
6 18 3
7 23 4
8 30 5
8 30 6
9 27 7
9 27 8
9 27 9
9 27 10
9 27 11
10 30 12
10 30 13
10 30 14
10 30 15
10 30 16
10 30 17
11 37 18
11 35 19
11 33 20
11 33 21
11 33 22
11 33 23
11 33 24
11 33 25
11 33 26
11 33 27
11 33 28
11 33 29
11 33 30
11 33 31
11 33 32
11 33 33
11 33 34
11 33 35
11 33 36
11 33 37
11 33 38
11 33 39
11 33 40
12 44 41
12 40 42
13 53 43
13 53 44
13 53 45
14 60 46
14 60 47
14 60 48
14 54 49
14 50 50
14 50 51
14 50 52
14 50 53
14 50 54
14 48 55
14 46 56
14 46 57
14 46 58
14 46 59
14 46 60
15 53 61
15 53 62
15 51 63
15 49 64
15 49 65
15 49 66
15 49 67
15 49 68
15 49 69
15 49 70
15 49 71
15 49 72
15 49 73
15 49 74
15 49 75
15 49 76
15 49 77
16 60 78
16 52 79
16 52 80
16 52 81
16 52 82
16 52 83
16 52 84
16 52 85
17 59 86
17 59 87
17 59 88
17 59 89
17 59 90
17 59 91
17 59 92
18 66 93
18 66 94
18 66 95
18 66 96
18 66 97
18 66 98
18 66 99
18 66 100
retrain  1 :
	train: 1.0 53.537798364400416 	test: 0.9090909090909091 24.480937134019904
retrain  2 :
	train: 1.0 39.85

KeyboardInterrupt: 

In [34]:
learning_rate=0.1
max_depth=10
bins=8
lam=1

gtgp = GTGP(learning_rate=learning_rate,max_depth=max_depth,bins=bins,lam=lam)

total_size=10
elite_size = 10
epoch=2000
gp_epoch=10
verbose = 1
re_train_epoch = 20
tolerance=0.0001

gtgp.fit(X_train,y_train,total_size=total_size,elite_size = elite_size,epoch=epoch,gp_epoch=gp_epoch,tolerance=tolerance,verbose=verbose)

57 305 1
127 729 2
179 961 3
217 1209 4
255 1377 5
296 1602 6
339 1815 7
372 1984 8
412 2228 9
448 2414 10
491 2667 11
529 2871 12
558 3034 13
589 3175 14
618 3364 15
644 3484 16
680 3692 17
718 3860 18
737 4003 19
781 4215 20
809 4399 21
837 4563 22
875 4783 23
906 4976 24
943 5187 25
959 5231 26
980 5374 27
1015 5591 28
1038 5674 29
1075 5843 30
1103 6027 31
1137 6361 32
1162 6530 33
1197 6755 34
1226 6936 35
1267 7255 36
1287 7355 37
1313 7517 38
1336 7660 39
1368 7856 40
1401 8081 41
1426 8192 42
1447 8311 43
1479 8537 44
1510 8724 45
1536 8874 46
1566 9070 47
1586 9152 48
1603 9231 49
1626 9332 50
1652 9492 51
1682 9672 52
1705 9821 53
1736 10016 54
1757 10129 55
1782 10280 56
1809 10451 57
1828 10558 58
1851 10673 59
1881 10843 60
1900 10930 61
1914 11010 62
1935 11143 63
1955 11243 64
1974 11366 65
1989 11441 66
2008 11582 67
2023 11653 68
2051 11869 69
2066 11932 70
2093 12093 71
2113 12227 72
2132 12356 73
2154 12520 74
2181 12691 75
2199 12795 76
2215 12865 77
2227 12929 78
2

In [ ]:
retrain_epoch=30
alpha=1
beta=1
gammer=0

verbose=1
gtgp.lam = 1
train_acc,test_acc,train_sse,test_sse = gtgp.retrain_estimators(X_test,y_test,retrain_epoch=retrain_epoch,alpha=alpha,beta=beta,gammer=gammer,verbose=verbose)


# verbose=0
# gtgp.retrain_estimators(retrain_epoch=retrain_epoch,alpha=alpha,beta=beta,gammer=gammer,verbose=verbose)

retrain  1 :
	train: 0.784918268519106 14296.751469714116 	test: 0.7819222419577659 6216.069217129345
retrain  2 :
	train: 0.7991076148787244 13328.290914812329 	test: 0.7947010065127295 5855.7129043267505


KeyboardInterrupt: 

In [ ]:
gtgp.print_model()

print(classification_report(y_train,np.argmax(gtgp.train_p,axis=1),digits=5))
print(classification_report(y_test,np.argmax(gtgp.test_p,axis=1),digits=5))

----------------GTGP-------------
Number of Trees: 4790
Average of depth: 2.1217118997912316
Number of nodes: 29108
              precision    recall  f1-score   support

           0    0.70522   0.22153   0.33715      4514
           1    0.79308   0.80153   0.79728     11644
           2    0.87025   0.95333   0.90989     31131

    accuracy                        0.84610     47289
   macro avg    0.78952   0.65880   0.68144     47289
weighted avg    0.83549   0.84610   0.82749     47289

              precision    recall  f1-score   support

           0    0.55904   0.15659   0.24465      1935
           1    0.75735   0.77419   0.76568      4991
           2    0.86023   0.94289   0.89966     13342

    accuracy                        0.82628     20268
   macro avg    0.72554   0.62456   0.63666     20268
weighted avg    0.80614   0.82628   0.80414     20268



# Decision Tree

In [7]:
from sklearn.tree import DecisionTreeClassifier

In [8]:
for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
    clf = DecisionTreeClassifier()
    clf.fit(X_train,y_train)

    num_trees = 1
    depth = clf.tree_.max_depth
    num_nodes = clf.tree_.node_count

    train_acc = accuracy_score(y_train,clf.predict(X_train))
    test_acc = accuracy_score(y_test,clf.predict(X_test))
    train_f1 = f1_score(y_train,clf.predict(X_train),average='macro')
    test_f1 = f1_score(y_test,clf.predict(X_test),average='macro')

    with open('./benchmark_DC/'+dataset+'.csv','a') as f:
        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

# XGBoost

In [13]:
import xgboost

In [27]:
for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
    xgb = xgboost.XGBClassifier(n_estimators=100)
    # xgb = xgboost.XGBClassifier(min_child_weight=0,max_depth=4)
    # xgb = xgboost.XGBClassifier()
    xgb.fit(X_train,y_train)

    import json

    def item_generator(json_input, lookup_key):
        if isinstance(json_input, dict):
            for k, v in json_input.items():
                if k == lookup_key:
                    yield v
                else:
                    yield from item_generator(v, lookup_key)
        elif isinstance(json_input, list):
            for item in json_input:
                yield from item_generator(item, lookup_key)

    def tree_depth(json_text):
        json_input = json.loads(json_text)
        depths = list(item_generator(json_input, 'depth'))
        return max(depths) + 1 if len(depths) != 0 else 1

    train_acc = accuracy_score(y_train,xgb.predict(X_train))
    test_acc = accuracy_score(y_test,xgb.predict(X_test))
    train_f1 = f1_score(y_train,xgb.predict(X_train),average='macro')
    test_f1 = f1_score(y_test,xgb.predict(X_test),average='macro')

    booster = xgb.get_booster()

    tree_df = booster.trees_to_dataframe()
    depths = [tree_depth(x) for x in booster.get_dump(dump_format = "json")]
    num_trees = len(depths)
    depth = np.average(depths)
    num_nodes = len(tree_df)

    with open('./benchmark_xgb/'+dataset+'.csv','a') as f:
        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

# GDBT

In [19]:
from sklearn.ensemble import GradientBoostingClassifier

In [21]:
for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
    # clf = GradientBoostingClassifier(n_estimators=1000)
    clf = GradientBoostingClassifier()
    clf.fit(X_train,y_train)

    train_acc = accuracy_score(y_train,clf.predict(X_train))
    test_acc = accuracy_score(y_test,clf.predict(X_test))
    train_f1 = f1_score(y_train,clf.predict(X_train),average='macro')
    test_f1 = f1_score(y_test,clf.predict(X_test),average='macro')

    num_trees = len([ est for ests in clf.estimators_ for est in ests])
    depth = np.average([ max(1,est.tree_.max_depth) for ests in clf.estimators_ for est in ests])
    num_nodes = sum([ est.tree_.node_count for ests in clf.estimators_ for est in ests])

    with open('./benchmark_GBDT/'+dataset+'.csv','a') as f:
        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

# Random Forest

In [17]:
from sklearn.ensemble import RandomForestClassifier

In [18]:
for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
    rfc = RandomForestClassifier()
    rfc.fit(X_train,y_train)

    train_acc = accuracy_score(y_train,rfc.predict(X_train))
    test_acc = accuracy_score(y_test,rfc.predict(X_test))
    train_f1 = f1_score(y_train,rfc.predict(X_train),average='macro')
    test_f1 = f1_score(y_test,rfc.predict(X_test),average='macro')

    num_trees = len(rfc.estimators_)
    depth = np.average([est.tree_.max_depth for est in rfc.estimators_])
    num_nodes = sum([est.tree_.node_count for est in rfc.estimators_])

    with open('./benchmark_RF/'+dataset+'.csv','a') as f:
        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)